In [1]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease [1,581 B]
Hit:3 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:5 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:6 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Ign:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:10 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:12 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [85.6 kB]
Hit:13 http:

In [2]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2022-07-22 23:43:49--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  4.75MB/s    in 0.2s    

2022-07-22 23:43:49 (4.75 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

### Load Amazon Data into Spark DataFrame

In [4]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Grocery_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show(5)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   42521656|R26MV8D0KG6QI6|B000SAQCWC|     159713740|The Cravings Plac...|         Grocery|          5|            0|          0|   N|                Y|Using these for y...|As a family aller...| 2015-08-31|
|         US|   12049833|R1OF8GP57AQ1A0|B00509LVIQ|     138680402|Mauna Loa Macadam...|         Grocery|          5|    

In [11]:
rows_count = df.count()

In [10]:
print (rows_count)

2402458


### Create DataFrames to match tables

In [12]:
from pyspark.sql.functions import to_date
# Read in the Review dataset as a DataFrame
review_df = df.select(["review_id","customer_id","product_id","product_parent","product_title","review_date","star_rating", "helpful_votes","total_votes","vine","verified_purchase"])
review_df.show()

+--------------+-----------+----------+--------------+--------------------+-----------+-----------+-------------+-----------+----+-----------------+
|     review_id|customer_id|product_id|product_parent|       product_title|review_date|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+----------+--------------+--------------------+-----------+-----------+-------------+-----------+----+-----------------+
|R26MV8D0KG6QI6|   42521656|B000SAQCWC|     159713740|The Cravings Plac...| 2015-08-31|          5|            0|          0|   N|                Y|
|R1OF8GP57AQ1A0|   12049833|B00509LVIQ|     138680402|Mauna Loa Macadam...| 2015-08-31|          5|            0|          0|   N|                Y|
|R3VDC1QB6MC4ZZ|     107642|B00KHXESLC|     252021703|Organic Matcha Gr...| 2015-08-31|          5|            0|          0|   N|                N|
|R12FA3DCF8F9ER|    6042304|B000F8JIIC|     752728342|15oz Raspberry Ly...| 2015-08-31|          5|       

In [14]:
# Create the customers_table DataFrame
customers_df = df.groupby("customer_id").agg({"customer_id":"count"}).withColumnRenamed("count(customer_id)", "customer_count")
customers_df.show()

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   21294263|             1|
|    8619125|             1|
|   43626894|             1|
|   38209321|             4|
|   40568205|             1|
|   13480593|             1|
|   19505799|             1|
|   43150161|             2|
|   28802315|             2|
|   35329257|             1|
|   49101505|             1|
|    1117644|             1|
|   28377689|             1|
|   23493243|             1|
|   48670265|             1|
|   49041516|             3|
|   37499049|             2|
|   10920752|             1|
|    5627998|             1|
|     135423|             1|
+-----------+--------------+
only showing top 20 rows



In [17]:
# Create the products_table DataFrame and drop duplicates. 
products_df = review_df.select(["product_id","product_title"]).drop_duplicates(["product_id"])
products_df.show()

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B0000CDBQB|Chef Paul Seasoni...|
|B0000CE2OA|Cashew Butter, Sm...|
|B0000D1699|          Pu-Erh Tea|
|B0000D17G4| Polish Dill Pickles|
|B0000D17HO|              Trenne|
|B0000D89E8|Helen Grace Choco...|
|B0000D94PL|Sweet 'N Low Whit...|
|B0000D9N78|Saffron from Spai...|
|B0000DC3QJ|Angel Falls Mist ...|
|B0000DG4MH|Mr. Hogg's Hogg W...|
|B0000DG4MV|Potatoes, Dehydra...|
|B0000DG6XY|Mt. Sterling Raw ...|
|B0000DIGP0|One Dozen Long St...|
|B0000DJ6NI|Tootsie Pops-Asso...|
|B0000DJDK2|Sante Fe Farms Fa...|
|B0000GIORA|Bijol Annatto, .5...|
|B0000SWIPM|Butter Roasted an...|
|B0000TL6A4|Melissa's Pickled...|
|B0000TL6EK|Melissa's Fresh C...|
|B0000TLERY|Wild Huckleberry ...|
+----------+--------------------+
only showing top 20 rows



In [19]:
# Create the review_id_table DataFrame. 
# Convert the 'review_date' column to a date datatype with to_date("review_date", 'yyyy-MM-dd').alias("review_date")
review_id_df = review_df.select(["review_id","customer_id", "product_id","product_parent", to_date("review_date", 'yyyy-MM-dd').alias("review_date")])
review_id_df.show()

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R26MV8D0KG6QI6|   42521656|B000SAQCWC|     159713740| 2015-08-31|
|R1OF8GP57AQ1A0|   12049833|B00509LVIQ|     138680402| 2015-08-31|
|R3VDC1QB6MC4ZZ|     107642|B00KHXESLC|     252021703| 2015-08-31|
|R12FA3DCF8F9ER|    6042304|B000F8JIIC|     752728342| 2015-08-31|
| RTWHVNV6X4CNJ|   18123821|B004ZWR9RQ|     552138758| 2015-08-31|
| RIG9AWFOGRDVO|   23649464|B00AL6QBZ6|     681475449| 2015-08-31|
|R1S1XSG4ZCHDGS|   32778285|B00BCNSTRA|     578681693| 2015-08-31|
| RB15NBVY5ELVW|   46612941|B0089MM2BG|     350968436| 2015-08-31|
| R56358YM1ZJ7I|   31525399|B00Y1C9770|     729982780| 2015-08-31|
|R1ODXB3C9UP3NL|   19624355|B00J074W94|       2499702| 2015-08-31|
|R155ZMVLD5C9BP|   31910375|B0001VKKOO|     189938205| 2015-08-31|
|R2IXW43PUNYM0G|   12816533|B007TGH4CK|     398128262| 2015-08

In [20]:
# Create the vine_table. DataFrame
vine_df = df.select(["review_id","star_rating","helpful_votes","total_votes","vine","verified_purchase"])
vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R26MV8D0KG6QI6|          5|            0|          0|   N|                Y|
|R1OF8GP57AQ1A0|          5|            0|          0|   N|                Y|
|R3VDC1QB6MC4ZZ|          5|            0|          0|   N|                N|
|R12FA3DCF8F9ER|          5|            0|          0|   N|                Y|
| RTWHVNV6X4CNJ|          5|            0|          0|   N|                Y|
| RIG9AWFOGRDVO|          2|            1|          1|   N|                Y|
|R1S1XSG4ZCHDGS|          5|            1|          1|   N|                Y|
| RB15NBVY5ELVW|          5|            2|          2|   N|                Y|
| R56358YM1ZJ7I|          5|            0|          0|   N|                N|
|R1ODXB3C9UP3NL|          1|            1|          3|   N|     

In [21]:
# Store environmental variable
from getpass import getpass
password = getpass('Enter database password:')

# Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://databiz.c3x2frc4y9xs.us-east-1.rds.amazonaws.com:5432/amazon"
config = {"user":"postgres", 
          "password": password, 
          "driver":"org.postgresql.Driver"}

Enter database password:··········


In [22]:
# Write review_id_df to table in RDS
review_id_df.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

In [23]:
# Write products_df to table in RDS
# about 3 min
products_df.write.jdbc(url=jdbc_url, table='products_table', mode=mode, properties=config)

In [24]:
# Write customers_df to table in RDS
# 5 min 14 s
customers_df.write.jdbc(url=jdbc_url, table='customers_table', mode=mode, properties=config)

In [25]:
# Write vine_df to table in RDS
# 11 minutes
vine_df.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)